In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv
import re

In [ ]:
#初始化WebDriver
def init_driver():
    options = webdriver.ChromeOptions()
    # 不使用 headless 模式以顯示瀏覽器
    options.add_argument('--disable-gpu')
    return webdriver.Chrome(options=options)

#第一階段：抓取景點ID與地點
def scrape_spot_ids():
    url = "https://tw.trip.com/travel-guide/attraction/taiwan-100076/tourist-attractions"
    driver = init_driver()
    driver.get(url)

    spot_data = []
    max_pages = 100  # 設定爬取的最大頁數
    current_page = 1

    while current_page <= max_pages:
        #滑動頁面並確保加載完成
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)  # 延長等待時間

        #解析當前頁面內容
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        spots = soup.select('a.online-poi-item-card[href^="https://tw.trip.com/travel-guide/attraction/"]')

        #驗證是否找到 spots
        print(f"第 {current_page} 頁，找到 {len(spots)} 個景點項目")
        if not spots:
            break

        for spot in spots:
            #提取 title 和 href
            title = spot.get('title')
            href = spot.get('href')

            #提取 city 和 spot_id
            match = re.search(r'attraction/([^/]+)/([^?]+)', href)
            city = match.group(1)
            spot_id = match.group(2)
            time.sleep(1)
            #添加到結果
            spot_data.append([city, spot_id, title])
            print(f"抓取: {title}, {city}/{spot_id}")

        #處理下一頁
        next_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'button.ant-pagination-item-link>[aria-label="right"]'))
        )

        #滾動到按鈕可見位置
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)

        #使用 JavaScript 點擊按鈕，繞過遮擋問題
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)
        current_page += 1

    driver.quit()

    #寫入CSV
    with open('spot_id.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['city', 'spot_id', 'title'])
        writer.writerows(spot_data)

    print("抓取完成，已寫入 spot_id.csv")

if __name__ == "__main__":
    scrape_spot_ids()


In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import re
import time

# 初始化 WebDriver
def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    return webdriver.Chrome(options=options)

# 第二階段：抓取評論與評分
def scrape_comments():
    with open('C:/Users/Administrator/Downloads/SCR/spot_id-.csv', 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        spots = list(reader)

    driver = init_driver()
    all_comments = []

    for spot in spots:
        city = spot['city']
        spot_id = spot['spot_id']
        title = spot['title']
        url = f"https://tw.trip.com/travel-guide/attraction/{city}/{spot_id}"
        print(f"正在抓取景點：{title}, URL: {url}")
        driver.get(url)

        # 確認景點名稱是否正確
        try:
            h1_title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.basicName'))
            ).text
            if h1_title != title:
                print(f"名稱不符，跳過：{h1_title}")
                continue
        except Exception:
            print(f"無法找到標題，跳過：{title}")
            continue

        print("滾動頁面以加載評論區...")

        # 抓取評論內容
        comments_count = 0
        while True:  # 使用無限循環，直到無下一頁按鈕
            # 滾動頁面以觸發評論加載
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # 等待評論載入

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            reviews = soup.select('div.review-item')

            if not reviews:
                print(f"無評論可抓取：{title}")
                break

            for review in reviews:
                try:
                    rating = review.select_one('div.ovh > div.score-box > span.review_score').text
                    comment_content = review.select_one('div > a[alt]').get('alt', '')
                    date_time = review.select_one('div.ovh > span.create-time').text.replace('撰寫日期：', '').strip()

                    # 格式化日期
                    date_time = re.sub(r'(\\d+) 年 (\\d+) 月 (\\d+) 日', r'\1-\2-\3', date_time)

                    all_comments.append([city, spot_id, title, rating, comment_content, date_time])
                    comments_count += 1
                except Exception as e:
                    print(f"評論處理錯誤：{e}")

            # 切換到下一頁
            try:
                # 找到所有下一頁按鈕
                next_buttons = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'button.btn-next'))
                )

                # 如果有兩個按鈕，點擊第二個
                if len(next_buttons) >= 2:
                    next_button = next_buttons[1]
                # 如果只有一個按鈕，點擊唯一按鈕
                elif len(next_buttons) == 1:
                    next_button = next_buttons[0]
                else:
                    print("未找到下一頁按鈕，結束評論抓取")
                    break

                # 檢查按鈕是否可見和可點擊
                if not next_button.is_displayed() or not next_button.is_enabled():
                    print("下一頁按鈕不可用或不可見，結束評論抓取")
                    break

                # 多次嘗試點擊下一頁按鈕
                for attempt in range(3):
                    # 滾動到按鈕可見範圍
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)  # 確保滾動完成

                    # 嘗試點擊按鈕
                    next_button.click()
                    time.sleep(2)

                    # 檢查是否成功切換頁面
                    new_reviews = BeautifulSoup(driver.page_source, 'html.parser').select('div.review-item')
                    if new_reviews and new_reviews != reviews:
                        print("成功切換到下一頁")
                        break
                    else:
                        print(f"第 {attempt + 1} 次嘗試切換頁面失敗，重試中...")
                else:
                    print("多次嘗試後仍未切換頁面，結束評論抓取")
                    break

            except Exception as e:
                print(f"無法切換評論頁面，可能已到最後一頁：{e}")
                break

        print(f"完成景點：{title} 的評論抓取")

    driver.quit()

    # 寫入 CSV
    with open('comments.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['city', 'spot_id', 'title', 'rating', 'comment_content', 'date_time'])
        writer.writerows(all_comments)
    print("評論抓取完成，已存入 comments.csv")

if __name__ == "__main__":
    scrape_comments()


正在抓取景點：十八羅漢山, URL: https://tw.trip.com/travel-guide/attraction/kaohsiung/shiba-luohan-mountain-13452364
滾動頁面以加載評論區...
成功切換到下一頁
第 1 次嘗試切換頁面失敗，重試中...
第 2 次嘗試切換頁面失敗，重試中...
第 3 次嘗試切換頁面失敗，重試中...
多次嘗試後仍未切換頁面，結束評論抓取
完成景點：十八羅漢山 的評論抓取
正在抓取景點：大安森林公園, URL: https://tw.trip.com/travel-guide/attraction/taipei/da-an-forest-park-10523108
滾動頁面以加載評論區...
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
第 1 次嘗試切換頁面失敗，重試中...
第 2 次嘗試切換頁面失敗，重試中...
第 3 次嘗試切換頁面失敗，重試中...
多次嘗試後仍未切換頁面，結束評論抓取
完成景點：大安森林公園 的評論抓取
正在抓取景點：台北101大樓, URL: https://tw.trip.com/travel-guide/attraction/taipei/taipei-101-10758289
滾動頁面以加載評論區...
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下一頁
成功切換到下